In [7]:
import os

In [8]:
%pwd

'c:\\Users\\nico_\\Desktop\\MLOPS'

In [ ]:
#os.chdir("../")
#%pwd

In [14]:
os.chdir("/users/nico_/Desktop/MLOPS/datascienceproject")

In [15]:
%pwd

'c:\\users\\nico_\\Desktop\\MLOPS\\datascienceproject'

In [50]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [51]:
from src.datascienceproject.constants import *
from src.datascienceproject.utils.common import read_yaml, create_directories

In [52]:
class ConfigurationManager:
    def __init__(self,
                  config_filepath=CONFIG_FILE_PATH,
                  params_filepath=PARAMS_FILE_PATH,
                  schema_filepath=SCHEMA_FILE_PATH):

        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config


In [53]:
import urllib.request as request
from src.datascienceproject import logger
import zipfile

In [65]:
import shutil

In [68]:
# Component - Data Ingestion
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config

    # Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            #filename, headers = request.urlretrieve(
                #url = self.config.source_URL,
                #filename = self.config.local_data_file
            #)
            shutil.copy(self.config.source_URL, self.config.local_data_file)
            #logger.info(f"{filename} download! with following info : \n{headers}")
            logger.info(f"File copied successfully from {self.config.source_URL} to {self.config.local_data_file}")
        else:
            logger.info(f"File already exists")

    def extact_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [70]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    #data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-04-24 12:20:35,240: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-24 12:20:35,244: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-24 12:20:35,246: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-04-24 12:20:35,246: INFO: common: created directory at: artifacts]
[2025-04-24 12:20:35,252: INFO: common: created directory at: artifacts/data_ingestion]
[2025-04-24 12:20:35,252: INFO: 2437221122: File already exists]
